In [1]:
import os
import json
import sqlite3
from tqdm import tqdm

# 📁 1. Dossier où se trouvent les fichiers JSON
folder_path = "C:/Users/pierr/OneDrive/Bureau/B3/Projet_python/test"  # Change ça si nécessaire

# 📦 2. Connexion à SQLite
db_path = "C:/Users/pierr/OneDrive/Bureau/test.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# 🧱 3. Création des tables SQL
cursor.execute('''
CREATE TABLE IF NOT EXISTS playlists (
    id INTEGER PRIMARY KEY,
    name TEXT,
    collaborative TEXT,
    modified_at INTEGER,
    num_tracks INTEGER,
    num_albums INTEGER,
    num_followers INTEGER
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS tracks (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    playlist_id INTEGER,
    position INTEGER,
    name TEXT,
    artist_name TEXT,
    album_name TEXT,
    duration_ms INTEGER,
    FOREIGN KEY(playlist_id) REFERENCES playlists(id)
)
''')

conn.commit()

# 🧮 4. Parcours des fichiers JSON
json_files = [f for f in os.listdir(folder_path) if f.endswith(".json")]
print(f"{len(json_files)} fichiers trouvés dans {folder_path}")

for filename in tqdm(json_files, desc="Traitement des fichiers"):
    file_path = os.path.join(folder_path, filename)

    try:
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        for playlist in data.get("playlists", []):
            pid = playlist["pid"]

            # Insertion playlist
            cursor.execute("""
                INSERT OR IGNORE INTO playlists (
                    id, name, collaborative, modified_at, num_tracks, num_albums, num_followers
                ) VALUES (?, ?, ?, ?, ?, ?, ?)
            """, (
                pid,
                playlist.get("name"),
                playlist.get("collaborative"),
                playlist.get("modified_at"),
                playlist.get("num_tracks"),
                playlist.get("num_albums"),
                playlist.get("num_followers")
            ))

            # Insertion des morceaux
            for track in playlist.get("tracks", []):
                cursor.execute("""
                    INSERT INTO tracks (
                        playlist_id, position, name, artist_name, album_name, duration_ms
                    ) VALUES (?, ?, ?, ?, ?, ?)
                """, (
                    pid,
                    track.get("pos"),
                    track.get("track_name"),
                    track.get("artist_name"),
                    track.get("album_name"),
                    track.get("duration_ms")
                ))

        conn.commit()

    except Exception as e:
        print(f"Erreur avec {filename} : {e}")

# ✅ 5. Requête de vérification
cursor.execute("SELECT COUNT(*) FROM playlists")
nb_playlists = cursor.fetchone()[0]
cursor.execute("SELECT COUNT(*) FROM tracks")
nb_tracks = cursor.fetchone()[0]

print(f"\n🎉 Import terminé : {nb_playlists} playlists et {nb_tracks} morceaux insérés.")

conn.close()


1 fichiers trouvés dans C:/Users/pierr/OneDrive/Bureau/B3/Projet_python/test


Traitement des fichiers: 100%|██████████| 1/1 [00:00<00:00, 821.29it/s]


🎉 Import terminé : 0 playlists et 0 morceaux insérés.
